# Notebook 03: DFT Setup Fundamentals

## Making the Right Choices Before Calculations

This notebook covers the critical decisions you must make before running DFT calculations:
- Exchange-correlation functional
- Pseudopotential selection
- Cutoff energies
- K-point sampling

---

## 1. Exchange-Correlation Functionals

### The Hierarchy

| Functional | Accuracy | Cost | When to Use |
|------------|----------|------|-------------|
| **LDA** | Low | Low | Qualitative trends, metals |
| **GGA (PBE)** | Medium | Low | Standard for most calculations |
| **GGA+U** | Medium-High | Low | Strongly correlated (d/f electrons) |
| **meta-GGA (SCAN)** | High | Medium | Better energetics |
| **Hybrid (HSE)** | High | High | Accurate band gaps |

### Typical Errors

| Property | LDA | GGA (PBE) |
|----------|-----|----------|
| Lattice parameter | -1 to -3% | +1 to +3% |
| Band gap | Underestimate | Underestimate |
| Formation energy | Overbinding | Better |

In [ ]:
# =============================================================================
# SETUP - Run this cell first
# =============================================================================

import sys
from pathlib import Path

_cwd = Path.cwd().resolve()
if (_cwd / 'workshop_setup.py').exists():
    sys.path.insert(0, str(_cwd))
elif (_cwd / 'notebooks_enhanced' / 'workshop_setup.py').exists():
    sys.path.insert(0, str(_cwd / 'notebooks_enhanced'))
else:
    raise FileNotFoundError("Cannot find workshop_setup.py. Run from the workshop root or notebooks_enhanced/")

from workshop_setup import *

from pymatgen.core import Structure, Lattice
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

import spglib

from ase import Atoms
from ase.build import bulk
from ase.io import read, write

print_workshop_banner()

print("\nSSP Cutoff Recommendations (from SSSP v1.3 Efficiency):")
print("=" * 60)
print(f"{'Element':<10} {'ecutwfc (Ry)':<15} {'ecutrho factor':<15}")
print("-" * 40)
for elem in ['Si', 'Al', 'Fe', 'O', 'Ti', 'Sr', 'Cu', 'N']:
    if elem in SSSP_EFFICIENCY:
        ecut, dual, _ = SSSP_EFFICIENCY[elem]
        print(f"{elem:<10} {ecut:<15} {dual}x")
print("=" * 60)

---

## Visualizing the Electron Density - The Heart of DFT!

**This is DENSITY Functional Theory** - the electron density $n(\mathbf{r})$ is the central quantity!

Unlike molecules where we visualize HOMO/LUMO orbitals, in solids we visualize:
- **Total charge density** $n(\mathbf{r})$ - Shows bonding character
- **Charge density difference** $\Delta n = n_{solid} - n_{atoms}$ - Shows charge redistribution
- **Partial charge density** - Charge from specific bands/energy ranges

### Visualization Workflow

```
SCF calculation → pp.x (post-processing) → .cube file → VESTA/XCrySDen
```

### What You'll See
- **Covalent bonds**: Charge accumulation between atoms (Si, C)
- **Ionic bonds**: Charge on anions, depleted on cations (NaCl)
- **Metallic bonds**: Uniform "sea" of electrons (Al, Cu)

In [ ]:
# =============================================================================
# CHARGE DENSITY VISUALIZATION - The Heart of DFT!
# =============================================================================
# After ANY SCF calculation, you can visualize the electron density.
# This generates .cube files viewable in VESTA, XCrySDen, or py3Dmol.
# =============================================================================

def generate_pp_input_charge(prefix: str, outdir: str = './tmp',
                             output_file: str = 'charge.cube',
                             plot_type: str = 'charge') -> str:
    """
    Generate pp.x input for charge density visualization.
    
    Parameters
    ----------
    prefix : str
        Must match SCF calculation prefix
    plot_type : str
        'charge' - Total charge density
        'spin'   - Spin polarization (nspin=2 only)
        'potential' - Local potential
    
    Returns
    -------
    str : pp.x input file content
    """
    plot_num = {'charge': 0, 'spin': 6, 'potential': 1, 'elf': 8}
    
    return f"""&INPUTPP
    prefix = '{prefix}'
    outdir = '{outdir}'
    filplot = '{prefix}.pp'
    plot_num = {plot_num.get(plot_type, 0)}
/

&PLOT
    nfile = 1
    filepp(1) = '{prefix}.pp'
    weight(1) = 1.0
    iflag = 3
    output_format = 6
    fileout = '{output_file}'
/
"""


def generate_pp_input_partial_charge(prefix: str, outdir: str = './tmp',
                                     emin: float = -5.0, emax: float = 0.0,
                                     output_file: str = 'partial_charge.cube') -> str:
    """
    Generate pp.x input for partial charge density (specific energy range).
    
    Useful for visualizing:
    - Valence band states (emax = Fermi level)
    - Conduction band states (emin = Fermi level)
    - States near band edges
    """
    return f"""&INPUTPP
    prefix = '{prefix}'
    outdir = '{outdir}'
    filplot = '{prefix}_partial.pp'
    plot_num = 7
    emin = {emin}
    emax = {emax}
/

&PLOT
    nfile = 1
    filepp(1) = '{prefix}_partial.pp'
    weight(1) = 1.0
    iflag = 3
    output_format = 6
    fileout = '{output_file}'
/
"""


def run_charge_density_visualization(prefix: str, work_dir, nprocs: int = 1):
    """
    Run pp.x to generate charge density cube file after SCF.
    
    Returns path to .cube file for VESTA visualization.
    """
    pp_file = work_dir / f'{prefix}_pp.in'
    cube_file = work_dir / f'{prefix}_charge.cube'
    
    pp_input = generate_pp_input_charge(prefix, output_file=str(cube_file))
    
    with open(pp_file, 'w') as f:
        f.write(pp_input)
    
    print(f"Running pp.x to generate charge density...")
    output, elapsed, _ = run_qe(pp_file, executable='pp.x', nprocs=nprocs)
    
    if cube_file.exists():
        print(f"✓ Charge density saved to: {cube_file}")
        print(f"\nTo visualize:")
        print(f"  1. Open VESTA")
        print(f"  2. File → Open → Select {cube_file.name}")
        print(f"  3. Use 'Isosurfaces' to see 3D density")
        return cube_file
    else:
        print("⚠ Cube file not generated - check pp.x output")
        return None


# Example pp.x inputs
print("=" * 70)
print("CHARGE DENSITY VISUALIZATION - pp.x Input Examples")
print("=" * 70)

print("\n1. Total Charge Density (after SCF):")
print("-" * 50)
print(generate_pp_input_charge('silicon'))

print("\n2. Partial Charge (valence states near Fermi level):")
print("-" * 50)
print(generate_pp_input_partial_charge('silicon', emin=-5.0, emax=0.0))

print("\n" + "=" * 70)
print("VIEWING IN VESTA:")
print("=" * 70)
print("""
1. Open VESTA and load the .cube file
2. Go to Properties → Isosurfaces
3. Set isosurface level (try 0.01-0.05 e/Å³ for charge)
4. Use 'Boundary' to extend visualization across unit cells
5. For bonding analysis: compare total vs. atomic superposition
""")

---

## 2. Pseudopotential Selection

### Types of Pseudopotentials

| Type | ecutrho | Accuracy | Speed | Use Case |
|------|---------|----------|-------|----------|
| **NC** (Norm-conserving) | 4× ecutwfc | Good | Fast | Quick tests, light elements |
| **US** (Ultrasoft) | 8-12× ecutwfc | Good | Medium | General purpose |
| **PAW** (Projector Augmented Wave) | 8× ecutwfc | Best | Slower | Production, all-electron precision |

### Recommended Libraries

1. **SSSP** (Standard Solid-State Pseudopotentials) - Materials Cloud
   - Efficiency set: Fast calculations
   - Precision set: High accuracy
   - URL: https://www.materialscloud.org/discover/sssp

2. **PSlibrary** - QE standard library
   - URL: https://dalcorso.github.io/pslibrary/

In [ ]:
# Pseudopotential information
SSSP_RECOMMENDATIONS = {
    # Element: (ecutwfc, dual, PP_file)
    'H':  (60, 8, 'H.pbe-rrkjus_psl.1.0.0.UPF'),
    'C':  (50, 8, 'C.pbe-n-kjpaw_psl.1.0.0.UPF'),
    'N':  (80, 8, 'N.pbe-n-kjpaw_psl.1.0.0.UPF'),
    'O':  (75, 8, 'O.pbe-n-kjpaw_psl.1.0.0.UPF'),
    'Si': (40, 8, 'Si.pbe-n-kjpaw_psl.1.0.0.UPF'),
    'Fe': (90, 12, 'Fe.pbe-spn-kjpaw_psl.1.0.0.UPF'),
    'Ti': (60, 8, 'Ti.pbe-spn-kjpaw_psl.1.0.0.UPF'),
    'Sr': (40, 8, 'Sr.pbe-spn-kjpaw_psl.1.0.0.UPF'),
    'Ba': (35, 8, 'Ba.pbe-spn-kjpaw_psl.1.0.0.UPF'),
}

def get_recommended_cutoffs(elements):
    """
    Get recommended cutoffs for a set of elements.
    Returns the maximum ecutwfc and appropriate ecutrho.
    """
    max_ecutwfc = 0
    max_dual = 4
    
    for elem in elements:
        if elem in SSSP_RECOMMENDATIONS:
            ecutwfc, dual, _ = SSSP_RECOMMENDATIONS[elem]
            max_ecutwfc = max(max_ecutwfc, ecutwfc)
            max_dual = max(max_dual, dual)
        else:
            print(f"Warning: {elem} not in database, using defaults")
            max_ecutwfc = max(max_ecutwfc, 60)
    
    return max_ecutwfc, max_ecutwfc * max_dual

# Example
elements = ['Sr', 'Ti', 'O']
ecutwfc, ecutrho = get_recommended_cutoffs(elements)
print(f"\nRecommended cutoffs for SrTiO3:")
print(f"  ecutwfc = {ecutwfc} Ry")
print(f"  ecutrho = {ecutrho} Ry")
print(f"\nNote: ALWAYS verify with convergence tests!")

---

## 3. K-point Sampling

### Rules of Thumb

- **Metals**: Dense grids needed (12×12×12 or more)
- **Semiconductors**: Moderate (6×6×6 to 8×8×8)
- **Insulators**: Coarser grids sufficient (4×4×4 to 6×6×6)
- **Large cells**: Fewer k-points needed (k × L ≈ constant)

### K-point Density

A useful metric is k-points per reciprocal atom (KPRA):
- ~1000 KPRA for insulators
- ~3000 KPRA for semiconductors
- ~5000+ KPRA for metals

In [ ]:
def estimate_kpoints(lattice_params, n_atoms, material_type='semiconductor'):
    """
    Estimate k-point grid based on lattice parameters and material type.
    
    Parameters
    ----------
    lattice_params : tuple
        (a, b, c) in Angstrom
    n_atoms : int
        Number of atoms in unit cell
    material_type : str
        'metal', 'semiconductor', or 'insulator'
    
    Returns
    -------
    tuple : (kx, ky, kz)
    """
    a, b, c = lattice_params
    
    # Target k-point density (reciprocal Å⁻¹)
    densities = {
        'metal': 0.03,
        'semiconductor': 0.05,
        'insulator': 0.07
    }
    target_spacing = densities.get(material_type, 0.05)
    
    # k_i ≈ 1 / (a_i × spacing)
    kx = max(1, int(np.ceil(1 / (a * target_spacing))))
    ky = max(1, int(np.ceil(1 / (b * target_spacing))))
    kz = max(1, int(np.ceil(1 / (c * target_spacing))))
    
    # Make odd for better symmetry
    kx = kx if kx % 2 == 1 else kx + 1
    ky = ky if ky % 2 == 1 else ky + 1
    kz = kz if kz % 2 == 1 else kz + 1
    
    return (kx, ky, kz)

# Examples
print("K-point Grid Estimation")
print("=" * 50)

materials = [
    ('Si (5.43 Å)', (5.43, 5.43, 5.43), 2, 'semiconductor'),
    ('Fe (2.87 Å)', (2.87, 2.87, 2.87), 2, 'metal'),
    ('MgO (4.21 Å)', (4.21, 4.21, 4.21), 2, 'insulator'),
]

for name, params, natoms, mtype in materials:
    kpts = estimate_kpoints(params, natoms, mtype)
    print(f"\n{name} ({mtype}):")
    print(f"  Suggested grid: {kpts[0]}×{kpts[1]}×{kpts[2]}")

---

## 4. Complete Input File Generator

In [1]:
def generate_scf_input(prefix, ecutwfc, ecutrho, kpoints, pseudo_dir,
                       celldm1=None, cell_parameters=None,
                       atomic_species=None, atomic_positions=None,
                       nspin=1, starting_magnetization=None,
                       hubbard_u=None, conv_thr=1.0e-8):
    """
    Generate a complete SCF input file for Quantum ESPRESSO.
    
    This function creates properly formatted input files following
    best practices for DFT calculations.
    """
    lines = []
    
    # CONTROL namelist
    lines.append("&CONTROL")
    lines.append("    calculation = 'scf'")
    lines.append(f"    prefix = '{prefix}'")
    lines.append("    outdir = './tmp'")
    lines.append(f"    pseudo_dir = '{pseudo_dir}'")
    lines.append("    verbosity = 'high'")
    lines.append("    tprnfor = .true.")
    lines.append("    tstress = .true.")
    lines.append("/")
    lines.append("")
    
    # SYSTEM namelist
    lines.append("&SYSTEM")
    
    if celldm1 is not None:
        lines.append("    ibrav = 0")  # Use CELL_PARAMETERS
        lines.append(f"    celldm(1) = {celldm1}")
    else:
        lines.append("    ibrav = 0")
    
    if atomic_species:
        nat = sum(len([p for p in atomic_positions if p[0] == s[0]]) 
                  for s in atomic_species) if atomic_positions else 2
        ntyp = len(atomic_species)
    else:
        nat = 2
        ntyp = 1
    
    lines.append(f"    nat = {nat}")
    lines.append(f"    ntyp = {ntyp}")
    lines.append(f"    ecutwfc = {ecutwfc}")
    lines.append(f"    ecutrho = {ecutrho}")
    lines.append("    occupations = 'smearing'")
    lines.append("    smearing = 'cold'")
    lines.append("    degauss = 0.01")
    
    if nspin == 2:
        lines.append(f"    nspin = {nspin}")
        if starting_magnetization:
            for i, mag in enumerate(starting_magnetization, 1):
                lines.append(f"    starting_magnetization({i}) = {mag}")
    
    if hubbard_u:
        lines.append("    lda_plus_u = .true.")
        for i, u in enumerate(hubbard_u, 1):
            lines.append(f"    Hubbard_U({i}) = {u}")
    
    lines.append("/")
    lines.append("")
    
    # ELECTRONS namelist
    lines.append("&ELECTRONS")
    lines.append(f"    conv_thr = {conv_thr}")
    lines.append("    mixing_beta = 0.7")
    lines.append("/")
    lines.append("")
    
    # ATOMIC_SPECIES
    lines.append("ATOMIC_SPECIES")
    if atomic_species:
        for symbol, mass, pp_file in atomic_species:
            lines.append(f"    {symbol}  {mass}  {pp_file}")
    else:
        lines.append("    Si  28.0855  Si.upf")
    lines.append("")
    
    # CELL_PARAMETERS
    if cell_parameters is not None:
        lines.append("CELL_PARAMETERS {angstrom}")
        for vec in cell_parameters:
            lines.append(f"    {vec[0]:12.8f}  {vec[1]:12.8f}  {vec[2]:12.8f}")
        lines.append("")
    
    # ATOMIC_POSITIONS
    lines.append("ATOMIC_POSITIONS {crystal}")
    if atomic_positions:
        for symbol, x, y, z in atomic_positions:
            lines.append(f"    {symbol}  {x:12.8f}  {y:12.8f}  {z:12.8f}")
    else:
        lines.append("    Si  0.00  0.00  0.00")
        lines.append("    Si  0.25  0.25  0.25")
    lines.append("")
    
    # K_POINTS
    kx, ky, kz = kpoints if isinstance(kpoints, tuple) else (kpoints, kpoints, kpoints)
    lines.append("K_POINTS {automatic}")
    lines.append(f"    {kx} {ky} {kz} 0 0 0")
    
    return '\n'.join(lines)

# Example: Generate input for SrTiO3
print("Example: SrTiO3 SCF Input")
print("=" * 60)

srtio3_input = generate_scf_input(
    prefix='srtio3',
    ecutwfc=75,
    ecutrho=600,
    kpoints=(6, 6, 6),
    pseudo_dir='./pseudopotentials',
    cell_parameters=[
        [3.905, 0.0, 0.0],
        [0.0, 3.905, 0.0],
        [0.0, 0.0, 3.905]
    ],
    atomic_species=[
        ('Sr', 87.62, 'Sr.pbe-spn-kjpaw_psl.1.0.0.UPF'),
        ('Ti', 47.867, 'Ti.pbe-spn-kjpaw_psl.1.0.0.UPF'),
        ('O', 15.999, 'O.pbe-n-kjpaw_psl.1.0.0.UPF')
    ],
    atomic_positions=[
        ('Sr', 0.0, 0.0, 0.0),
        ('Ti', 0.5, 0.5, 0.5),
        ('O', 0.5, 0.5, 0.0),
        ('O', 0.5, 0.0, 0.5),
        ('O', 0.0, 0.5, 0.5)
    ]
)

print(srtio3_input)

Example: SrTiO3 SCF Input
&CONTROL
    calculation = 'scf'
    prefix = 'srtio3'
    outdir = './tmp'
    pseudo_dir = './pseudopotentials'
    verbosity = 'high'
    tprnfor = .true.
    tstress = .true.
/

&SYSTEM
    ibrav = 0
    nat = 5
    ntyp = 3
    ecutwfc = 75
    ecutrho = 600
    occupations = 'smearing'
    smearing = 'cold'
    degauss = 0.01
/

&ELECTRONS
    conv_thr = 1e-08
    mixing_beta = 0.7
/

ATOMIC_SPECIES
    Sr  87.62  Sr.pbe-spn-kjpaw_psl.1.0.0.UPF
    Ti  47.867  Ti.pbe-spn-kjpaw_psl.1.0.0.UPF
    O  15.999  O.pbe-n-kjpaw_psl.1.0.0.UPF

CELL_PARAMETERS {angstrom}
      3.90500000    0.00000000    0.00000000
      0.00000000    3.90500000    0.00000000
      0.00000000    0.00000000    3.90500000

ATOMIC_POSITIONS {crystal}
    Sr    0.00000000    0.00000000    0.00000000
    Ti    0.50000000    0.50000000    0.50000000
    O    0.50000000    0.50000000    0.00000000
    O    0.50000000    0.00000000    0.50000000
    O    0.00000000    0.50000000    0.50000

---

## 5. Common Mistakes to Avoid

| Mistake | Consequence | Solution |
|---------|-------------|----------|
| Wrong PP type | Incorrect ecutrho | Check PP documentation |
| Insufficient ecutwfc | Unconverged energies | Run convergence test |
| Too few k-points | Wrong energetics | Increase grid for metals |
| Missing +U for d/f | Wrong electronic structure | Add Hubbard U |
| Smearing for insulators | Artificial metallicity | Use 'tetrahedra' or small degauss |

---

## Summary

Before running calculations, verify:

1. ✓ **Functional**: PBE for most, +U for d/f metals, HSE for gaps
2. ✓ **Pseudopotentials**: SSSP library, correct type (NC/US/PAW)
3. ✓ **Cutoffs**: Use PP recommendations, verify with convergence
4. ✓ **K-points**: Dense for metals, moderate for semiconductors

### Next Notebook
→ **04_Convergence_Testing.ipynb**: Systematic convergence tests